## Skorch RHC network classification example

In [ ]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch' , 'torch'])
except ImportError:
    pass

In [1]:
import numpy as np
from sklearn.datasets import make_classification
import torch
from torch import nn
from skorch import NeuralNetClassifier
from skorch import NeuralNet
from pyperch.neural.rhc_nn import RHCModule  
from pyperch.utils.decorators import add_to
from skorch.dataset import unpack_data
import copy

In [2]:
X, y = make_classification(1000, 12, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)
print(X.shape, y.shape)
input_dim=12
output_dim=2
hidden_units=20

(1000, 12) (1000,)


In [3]:
rhc_module=RHCModule(input_dim=input_dim, output_dim=output_dim, hidden_units=hidden_units)

net = NeuralNetClassifier(
    rhc_module,
    max_epochs=500,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

## train_step_single override - add RHC training step and disable backprop 

In [4]:
RHCModule.register_rhc_training_step()

In [5]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6891       0.5300        0.6982  0.0224
      2        0.6877       0.5350        0.6969  0.0255
      3        0.6858       0.5400        0.6948  0.0228
      4        0.6851       0.5350        0.6948  0.0339
      5        0.6841       0.5350        0.6919  0.0277
      6        0.6821       0.5500        0.6909  0.0321
      7        0.6800       0.5500        0.6882  0.0257
      8        0.6794       0.5650        0.6870  0.0312
      9        0.6774       0.5850        0.6837  0.0230
     10        0.6753       0.5850        0.6813  0.0267
     11        0.6740       0.5900        0.6792  0.0306
     12        0.6710       0.5900        0.6781  0.0292
     13        0.6699       0.5950        0.6776  0.0330
     14        0.6695       0.5900        0.6782  0.0277
     15        0.6683       0.5900        0.6785  0.0283
     16        0.6684       0.5

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=RHCModule(
    (dense0): Linear(in_features=12, out_features=20, bias=True)
    (nonlin): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=20, out_features=20, bias=True)
    (output): Linear(in_features=20, out_features=2, bias=True)
    (softmax): Softmax(dim=-1)
  ),
)

## Using sklearn pipeline with RO

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6898       0.5050        0.6940  0.0260
      2        0.6899       0.5000        0.6937  0.0257
      3        0.6897       0.4950        0.6938  0.0255
      4        0.6892       0.5050        0.6931  0.0244
      5        0.6887       0.5150        0.6917  0.0200
      6        0.6878       0.5250        0.6913  0.0247
      7        0.6871       0.5300        0.6901  0.0199
      8        0.6867       0.5350        0.6891  0.0265
      9        0.6861       0.5350        0.6891  0.0258
     10        0.6854       0.5250        0.6882  0.0237
     11        0.6848       0.5450        0.6883  0.0217
     12        0.6844       0.5400        0.6881  0.0249
     13        0.6843       0.5400        0.6881  0.0218
     14        0.6843       0.5500        0.6886  0.0247
     15   

## Using sklearn grid search with RO

In [8]:
from sklearn.model_selection import GridSearchCV

# deactivate skorch-internal train-valid split and verbose logging
net.set_params(train_split=False, verbose=0, )

default_params = {
    'module__input_dim': [12],
    'module__output_dim': [2],
}

grid_search_params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    'module__hidden_units': [10, 20],
    **default_params,
}

gs = GridSearchCV(net, grid_search_params, refit=False, cv=3, scoring='accuracy', verbose=2)

gs.fit(X, y)
print("best score: {:.3f}, best params: {}".format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END lr=0.01, max_epochs=10, module__hidden_units=10, module__input_dim=20, module__output_dim=2; total time=   0.0s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=10, module__input_dim=20, module__output_dim=2; total time=   0.0s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=10, module__input_dim=20, module__output_dim=2; total time=   0.0s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=20, module__input_dim=20, module__output_dim=2; total time=   0.0s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=20, module__input_dim=20, module__output_dim=2; total time=   0.0s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=20, module__input_dim=20, module__output_dim=2; total time=   0.0s
[CV] END lr=0.01, max_epochs=20, module__hidden_units=10, module__input_dim=20, module__output_dim=2; total time=   0.0s
[CV] END lr=0.01, max_epochs=20, module__hidden_units=10, module__input_dim=20, module__outpu

ValueError: 
All the 24 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/home/john/anaconda3/envs/pyperch/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/john/anaconda3/envs/pyperch/lib/python3.11/site-packages/skorch/classifier.py", line 165, in fit
    return super(NeuralNetClassifier, self).fit(X, y, **fit_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/john/anaconda3/envs/pyperch/lib/python3.11/site-packages/skorch/net.py", line 1319, in fit
    self.partial_fit(X, y, **fit_params)
  File "/home/john/anaconda3/envs/pyperch/lib/python3.11/site-pa